## This is a tool to support talk map

In [1]:
import requests
import numpy as np
from pybtex.database.input import bibtex
from bs4 import BeautifulSoup as bs
import os
import glob

In [2]:
with open("/Users/zongrunli/Desktop/HomePage/KEYS_GOOGLE.txt", "r") as f:
    API_KEY = f.read().strip()

In [3]:
def convertAddressBing(address, API_KEY=API_KEY):
    # The bounds parameter defines the latitude/longitude coordinates
    # of the southwest and northeast corners of this bounding box
    params = {
        'addressLine': address,
        'key': API_KEY,
        'strictMatch': 1,
    }
    baseurl = "http://dev.virtualearth.net/REST/v1/Locations?"
    res = requests.get(baseurl, params=params).json()
    if "errorDetails" in res.keys():
        print(res["errorDetails"])
        return np.NAN, np.NAN, np.NAN
    elif res['resourceSets'][0]['estimatedTotal'] == 0:
        return np.NAN, np.NAN, np.NAN
    else:
        lat, long = res['resourceSets'][0]['resources'][0]['point']['coordinates']
        found_address = res['resourceSets'][0]['resources'][0]["name"]
        return lat, long, found_address

def convertAddressGoogle(address, API_KEY=API_KEY):
    baseurl = "https://maps.googleapis.com/maps/api/geocode/json"

    params = {
        "address": address,
        "key": API_KEY
    }

    res = requests.get(baseurl, params=params).json()

    # Error case 1: Google API error
    if res.get("status") != "OK":
        print("Google API Error:", res.get("status"), res.get("error_message"))
        return np.nan, np.nan, np.nan

    # Extract first result
    result = res["results"][0]
    lat = result["geometry"]["location"]["lat"]
    lng = result["geometry"]["location"]["lng"]
    found_address = result["formatted_address"]

    return lat, lng, found_address

def conference_info(cite_name):
    parser = bibtex.Parser()
    bibdata = parser.parse_file(cite_name)
    conference_obj = None
    for bib_id in bibdata.entries:
        b = bibdata.entries[bib_id].fields
        conference_obj = {
            "title": b["title"],
            "conference": b["booktitle"],
            "year": b["year"],
            "location": b["location"]
        }
        author_list = []
        for author in bibdata.entries[bib_id].persons["author"]:
            author_names = []
            author_info = [author.first_names, author.middle_names, author.last_names]
            for i in author_info:
                if len(i) > 0:
                    author_names.append(i[0])
            author_name = " ".join(author_names)
            author_list.append(author_name)
        conference_obj["authors"] = author_list
    return conference_obj

def generate_html_text(cite_name, match_name="Zongrun Li"):
    conference_obj = conference_info(cite_name)
    html_text = """
                <li>
                    <div class="conference-title">
                        %s
                    </div>
            """ % conference_obj["title"]

    for i in range(0, len(conference_obj["authors"])):
        author = conference_obj["authors"][i]
        if author == match_name:
            html_text += "<strong>" + match_name + "</strong>"
        else:
            html_text += author
        
        if i != len(conference_obj["authors"]) - 1:
            html_text += ", "
    
    html_text += """
        <br><em>%s</em>, %s
        <br>
    </li>
    """% (conference_obj["conference"], conference_obj["year"])
    
    soup = bs(html_text)
    html_text = soup.prettify()
    return html_text

def generate_js_address_points(locations):
    js_lines = ["var addressPoints = ["]
    
    for (lon, lat), info in locations.items():
        name = info["name"]

        # generate HTML
        html_txt = ""
        for f in info["file"]:
            html_txt += generate_html_text(f)

        # escape JS special characters
        name_js = name.replace('"', '\\"')
        html_js = html_txt.replace('"', '\\"').replace('\n', '\\n')

        # build JS array line
        line = f'    ["{name_js}", {lat}, {lon}, "{html_js}"],'
        js_lines.append(line)

    js_lines.append("];")

    return "\n".join(js_lines)

In [4]:
# create JS
# ['Baltimore, MD, USA', 39.28870812, -76.63482396, '<div class="conference-title">Modeling the Regional Air Quality Impacts of Prescribed Burning at a Military Base in Southeastern United States</div><strong>Zongrun Li</strong>, <a>Yongtao Hu</a>, <a>M Talat Odman</a>, <a>Armistead G Russell</a><br><em>American Meteorological Society 103rd Annual Meeting</em>, 2024']
# process location and cluster locations
talks_folder_relative = "./talks"
sub_paths = glob.glob(os.path.join(talks_folder_relative, '*'))
locations = {}
for cur_path in sub_paths:
    location_name = conference_info(cur_path)["location"]
    lat, lon, bing_name = convertAddressGoogle(location_name)
    # locations.append((lon, lat))
    print("%s %s" % (location_name, bing_name))
    if (lon, lat) in locations.keys():
        locations[(lon, lat)]["file"].append(cur_path)
    else:
        locations[(lon, lat)] = {"file": [cur_path], "name": location_name}

George R. Brown Convention Center, Houston, Texas 1001 Avenida De Las Americas, Houston, TX 77010, USA
Walter E. Washington Convention Center 801 Allen Y. Lew Place NW Washington, D.C. 801 Allen Y. Lew Place NW, Washington, DC 20001, USA
New Orleans Ernest N. Morial Convention Center, Hall EFG, New Orleans, Louisiana 900 Convention Center Blvd, New Orleans, LA 70130, USA
Baltimore, MD Baltimore, MD, USA
Denver, CO Denver, CO, USA
Chapel Hill, NC Chapel Hill, NC, USA
Chapel Hill, NC Chapel Hill, NC, USA
Chapel Hill, NC Chapel Hill, NC, USA


In [5]:
# # generate js variable
# for location in locations.keys():
#     cur_lon, cur_lat = location
#     cur_info = locations[location]
#     cur_list = [cur_info["name"], cur_lat, cur_lon]
#     html_txt = ""
#     for file in cur_info["file"]:
#         cur_html = generate_html_text(file)
#         html_txt += cur_html
#     cur_list.append(html_txt)
#     print(cur_list, end=", \n")
js_code = generate_js_address_points(locations)
with open("org-locations.js", "w") as f:
    f.write(js_code)
